In [12]:
from nilearn import image, connectome, plotting
from nilearn.maskers import NiftiLabelsMasker
import os
import numpy as np
import matplotlib.pyplot as plt


Schaefer100Parcels = image.load_img('parcellation/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_2mm (1).nii')

nifti_dir = 'nifti_files'


In [ ]:
# Only run if recomputing and saving all FC matrices 

for filename in os.listdir(nifti_dir):
    nifti_file = os.path.join(nifti_dir, filename)

    subject_number = filename.split('-')[1].split('_')[0]

    # Create masker and extract time series
    masker = NiftiLabelsMasker(labels_img=Schaefer100Parcels, standardize='zscore_sample')
    time_series = masker.fit_transform(nifti_file)

    # Create correlation matrix
    correlation_measure = connectome.ConnectivityMeasure(kind='correlation', standardize='zscore_sample')
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]

    # Save as numpy file
    output_filename = os.path.join('fc_matrices/', f'fc_matrix_{subject_number}.npy')
    np.save(output_filename, correlation_matrix)

    # Plot the FC plots
    title = f'Functional Connectivity Matrix for subject {subject_number}'
    plot_filename = os.path.join('fc_matrix_plots', f'fc_matrix_plot_{subject_number}.png')
    plotting.plot_matrix(correlation_matrix, colorbar=True, figure=(8, 6), title=title, cmap='coolwarm')
    plt.savefig(plot_filename) 
    plt.close() 
    

In [13]:
# Only run if calculating the averaged FC matrix from already computed npy files

npy_dir = 'fc_matrices'
all_correlation_matrices = []

# Load and accumulate correlation matrices from all npy files
for filename in os.listdir(npy_dir):
    if filename.endswith('.npy'):
        file_path = os.path.join(npy_dir, filename)
        correlation_matrix = np.load(file_path)
        all_correlation_matrices.append(correlation_matrix)

# Compute the average FC matrix across all subjects
average_correlation_matrix = np.mean(all_correlation_matrices, axis=0)

output_filename = os.path.join('fc_matrices/', 'averaged_fc_matrix.npy')
np.save(output_filename, average_correlation_matrix)

title = 'Averaged Functional Connectivity Matrix'
plot_filename = os.path.join('fc_matrix_plots', 'averaged_fc_matrix_plot.png')
plotting.plot_matrix(average_correlation_matrix, colorbar=True, figure=(8, 6), title=title, cmap='coolwarm')
plt.savefig(plot_filename)
plt.close()